In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sam import SamPredictor, sam_model_registry
from sam.utils.transforms import ResizeLongestSide

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b_01ec64.pth")
image_encoder = sam.image_encoder
prompt_encoder = sam.prompt_encoder

In [ ]:
import clip
clip_model, preprocess = clip.load("ViT-B/32", device=device)
text = clip.tokenize(["brain"]).to(device)
text_features = clip_model.encode_text(text)

# 前處理

In [ ]:
# import json,os
# import numpy as np
# from monai.transforms import (
#     AsDiscrete,
#     Compose,
#     CropForegroundd,
#     LoadImaged,
#     Orientationd,
#     RandFlipd,
#     RandCropByPosNegLabeld,
#     RandShiftIntensityd,
#     ScaleIntensityRanged,
#     Spacingd,
#     RandRotate90d,
#     EnsureTyped,
#     SpatialCrop,
#     AddChanneld,
#     Transform,
#     ResizeWithPadOrCropd,
#     Lambda,
# )
# from monai.data import (
#     ThreadDataLoader,
#     CacheDataset,
#     load_decathlon_datalist,
#     decollate_batch,
#     set_track_meta,
#     DataLoader,
# )

# def get_evaluation_transform(spacing):
#     return Compose(
#         [
#             LoadImaged(keys=["image", "label"], ensure_channel_first=None),
#             AddChanneld(keys=["image", "label"]),
#             ScaleIntensityRanged(
#                 keys=["image"], a_min=-1024, a_max=3071, b_min=0.0, b_max=255.0, clip=True
#             ),
#             CropForegroundd(keys=["image", "label"], source_key="image"),
#             Orientationd(keys=["image", "label"], axcodes="RAS"),
#             Spacingd(
#                 keys=["image", "label"],
#                 pixdim=(spacing[0], spacing[1], spacing[2]),
#                 mode=("bilinear", "nearest"),
#             ),
#             EnsureTyped(keys=["image", "label"], device=device, track_meta=True),
#         ]
#     )

# def custom_load_decathlon_datalist(json_data_path, index):
#     with open(json_data_path, 'r', encoding='utf-8') as json_file:
#         json_data = json_file.read()
#     parsed_data = json.loads(json_data)
#     target = parsed_data[index]
#     # 路徑+去掉.變成完整路徑
#     for i in range(len(target)):
#         target[i]['image'] = data_dir + target[i]['image'][1:]
#         target[i]['label'] = data_dir + target[i]['label'][1:]

#     return target

# # image數量
# organ_num = {
#     3:131,
#     6:63,
#     7:281,
#     8:303,
#     9:41,
#     10:126,
# }
# spacings = {
#     2: [1.25, 1.25, 1.37],
#     3: [0.7676, 0.7676, 1],
#     6: [0.79, 0.79, 1.24],
#     7: [0.8, 0.8, 2.5],
#     8: [0.8, 0.8, 1.5],
#     9: [0.78, 0.78, 1.6],
#     10: [0.78, 0.78, 3],
# }
# data_dir ="D:\\SAM\\data"

In [ ]:
# import cv2
# task_list = [3,6,7,8,9,10] #3,6,7,8,9,10
# for task in task_list:
#     datasets = os.path.join(data_dir, f"dataset_{task}.json")

#     # 得到data path list
#     datalist = custom_load_decathlon_datalist(datasets, "training")

#     split_index = organ_num[task]* 4 // 5
#     segment1 = datalist[0:7]
#     segment2 = datalist[split_index:min(split_index+7, organ_num[task])]
#     # 连接两个段
#     eval_list = segment1 + segment2
#     print('eval_list', eval_list, len(eval_list))

#     dataset = CacheDataset(
#         data = eval_list,
#         transform = get_evaluation_transform(spacings[task]),
#         cache_num = 24,
#         cache_rate = 1.0,
#         num_workers = 8,
#     )   

#     # split_index = len(datalist) * 4 // 5

#     for i in range(len(dataset)):
#         image, label = dataset[i]['image'], dataset[i]['label']
        
#         # 獲取圖像和標籤的形狀
#         # print(type(image))
#         # print(image.shape)
#         _, D, H, W = image.shape
        
#         # 遍歷每一個slice
#         for d in range(D):
#             # 獲取2D slice
#             image_slice = image[0, d, :, :]
#             label_slice = label[0, d, :, :]
#             image_slice_uint8 = image_slice.astype(np.uint8)

#             # save as numpy
#             if not os.path.exists(f"my_np_data/task{task}"):
#                 os.makedirs(f"my_np_data/task{task}")
#                 os.makedirs(f"my_np_data/task{task}/image")
#                 os.makedirs(f"my_np_data/task{task}/label")
#                 os.makedirs(f"my_np_data/task{task}/gray")
#             np.save(f'my_np_data/task{task}/image/image_{i}_{d}.npy', image_slice.cpu().numpy())
#             np.save(f'my_np_data/task{task}/label/label_{i}_{d}.npy', label_slice.cpu().numpy())
#             np.save(f'my_np_data/task{task}/gray/image_{i}_{d}.npy', image_slice_uint8)

#         #     # 顯示圖像
#         #     cv2.imshow('Image Slice', image_slice_uint8)
#         #     cv2.imshow('Label Slice', label_slice.cpu().numpy())
            
#         #     # 等待並檢查是否有按鍵事件
#         #     key = cv2.waitKey(0) & 0xFF
            
#         #     # 如果按下 'q' 鍵，則退出循環
#         #     if key == ord('q'):
#         #         break
#         # # 如果按下 'q' 鍵，則退出循環
#         # if key == ord('q'):
#         #     break

# # cv2.destroyAllWindows()

# check npy data is ok

In [ ]:
# for i in range(457):
#     img = np.load(f'my_np_data/task3/image/image_0_{i}.npy')
#     gray = np.load(f'my_np_data/task3/gray/image_0_{i}.npy')
#     label = np.load(f'my_np_data/task3/label/label_0_{i}.npy')
#     cv2.imshow('image', img)
#     cv2.imshow('label', label)
#     cv2.imshow('gray', gray)
#     # 等待並檢查是否有按鍵事件
#     key = cv2.waitKey(0) & 0xFF

#     # 如果按下 'q' 鍵，則退出循環
#     if key == ord('q'):
#         break
# cv2.destroyAllWindows()

# data loader

In [ ]:
# import os
# import re
# from collections import defaultdict

# img_dir = 'my_np_data/task3/image'
# pattern = re.compile(r'image_(\d+)_(\d+)\.npy')

# # 初始化一個defaultdict，預設值為0
# slice_counts = defaultdict(int)

# for filename in os.listdir(img_dir):
#     match = pattern.match(filename)
#     if match:
#         I, _ = map(int, match.groups())
#         # 每找到一個符合的檔案，就將對應的I的計數加一
#         slice_counts[I] += 1

# print(slice_counts)


In [ ]:
nums_of_data = {
    3: { # task
        0: 457, # slice of ith image
        1: 503,
        2: 521,
        3: 496,
        4: 615,
        5: 417,
        6: 667,
        7: 667,
        8: 667,
        9: 552,
        10: 570,
        11: 512,
        12: 456,
        13: 665,
    },
    6: {
        0: 607,
        1: 546,
        2: 532,
        3: 510,
        4: 481,
        5: 631,
        6: 470,
        7: 557,
        8: 519,
        9: 506,
        10: 494,
        11: 449,
        12: 532,
        13: 506,
    },
    7: {
        0: 586,
        1: 556,
        2: 490,
        3: 488,
        4: 620,
        5: 463,
        6: 522,
        7: 550,
        8: 547,
        9: 471,
        10: 547,
        11: 485,
        12: 536,
        13: 450,
    },
    8: {
        0: 587,
        1: 609,
        2: 619,
        3: 604,
        4: 572,
        5: 540,
        6: 551,
        7: 450,
        8: 504,
        9: 469,
        10: 499,
        11: 450,
        12: 499,
        13: 463,
    },
    9: {
        0: 523,
        1: 486,
        2: 528,
        3: 492,
        4: 485,
        5: 403,
        6: 477,
        7: 515,
        8: 515,
        9: 604,
        10: 508,
        11: 631,
        12: 583,
        13: 631,
    },
    10: {
        0: 462,
        1: 512,
        2: 597,
        3: 530,
        4: 470,
        5: 540,
        6: 508,
        7: 503,
        8: 531,
        9: 495,
        10: 515,
        11: 513,
        12: 612,
        13: 531,
    },
}

# 將nums_of_data轉成累加的形式
for task, slices in nums_of_data.items():
    for i in range(1, len(slices)):
        slices[i] += slices[i - 1]

class MyDataset(Dataset):
    def __init__(self, data_path, task_num):
        self.img_dir = os.path.join(data_path, f"task{task_num}", "image")
        self.label_dir = os.path.join(data_path, f"task{task_num}", "label")
        self.task_num = task_num

    def __len__(self):
        return nums_of_data[self.task_num][13]

    def __getitem__(self, idx):
        i, d = self.find_image_and_slice(idx)
        print(f"{d}th slice of {i}th image")
        img_path = f'{self.img_dir}/image_{i}_{d}.npy'
        label_path = f'{self.label_dir}/label_{i}_{d}.npy'
        return torch.from_numpy(np.load(img_path)), torch.from_numpy(np.load(label_path))
    
    def find_image_and_slice(self, index):
        for i in range(len(nums_of_data[self.task_num])):
            if index < nums_of_data[self.task_num][i]:
                slice_index = index - nums_of_data[self.task_num][i - 1] if i > 0 else index
                return i, slice_index


task = 3
data_path = "my_np_data"
dataset = MyDataset(data_path, task)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from typing import List, Tuple, Type
import math
from einops import rearrange
class Text2ImageTransformer(nn.Module):
    def __init__(
        self,
        depth: int = 2,
        embedding_dim: int = 256,
        num_heads: int = 8,
        mlp_dim: int = 384,
        activation: Type[nn.Module] = nn.ReLU,
        attention_downsample_rate: int = 1,
    ) -> None:
        super().__init__()
        self.depth = depth
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.mlp_dim = mlp_dim
        self.align_layers = nn.Sequential(
            nn.Linear(512, embedding_dim),
            nn.ReLU(),
        )
        self.layers = nn.ModuleList()
        for _ in range(depth):
            self.layers.append(
                Text2ImageAttentionBlock(
                    embedding_dim=embedding_dim,
                    num_heads=num_heads,
                    mlp_dim=mlp_dim,
                    activation=activation,
                    attention_downsample_rate=attention_downsample_rate
                )
            )

    def forward(self, image_embedding, text_embedding) -> Tuple[Tensor, Tensor]:
        # image_embedding = [1,256,64,64]
        # text_embedding = [1,512]
        image_embedding = rearrange(image_embedding, 'b c h w -> b (h w) c')
        text_embedding = self.align_layers(text_embedding).unsqueeze(1)
        print("image_embedding supossed to be [1,4096,256]:", image_embedding.size())
        print("text_embedding supossed to be [1,1,256]:", text_embedding.size())
        for _, layer in enumerate(self.layers): # 2個block
            image_embedding, text_embedding = layer(
                image_embedding,
                text_embedding
            )

        # 此時image_embedding主要包含了圖像本身的資訊，而text_embedding則包含了圖像和文本之間的關聯資訊。
        return image_embedding, text_embedding
    
class Text2ImageAttentionBlock(nn.Module):
    def __init__(
        self,
        embedding_dim: int = 256,
        num_heads: int = 8,
        mlp_dim: int = 384,
        activation: Type[nn.Module] = nn.ReLU,
        attention_downsample_rate: int = 1,
    ) -> None:
        super().__init__()
        self.self_attn = Attention(embedding_dim, num_heads)
        self.norm1 = nn.LayerNorm(embedding_dim)
        self.cross_attn = Attention(embedding_dim, num_heads)
        self.norm2 = nn.LayerNorm(embedding_dim)
        self.mlp = MLPBlock(embedding_dim, mlp_dim, activation)
        self.norm3 = nn.LayerNorm(embedding_dim)
        # self.global_query = nn.parameter.Parameter(data=0.1 * torch.randn(1, 10, embedding_dim)) # [1,10,256]的可訓練參數(隨機產生的正態分布*0.1)

    def forward(self, image_embedding, text_embedding) -> Tensor:
        # self attention
        self_out = self.self_attn(q=image_embedding, k=image_embedding, v=image_embedding)
        self_out = self.norm1(image_embedding + self_out) # skip connection
        print("self_out", self_out.size())

        # cross attention
        cross_out = self.cross_attn(q=self_out, k=text_embedding, v=text_embedding)
        cross_out = self.norm2(self_out + cross_out) # skip connection
        print("cross_out", self_out.size())

        # MLP block
        mlp_out = self.mlp(cross_out)
        mlp_out = self.norm3(cross_out + mlp_out) # skip connection
        print("mlp_out", mlp_out.size())

        return self_out, mlp_out

class Attention(nn.Module):
    def __init__(
        self,
        embedding_dim: int = 256,
        num_heads: int = 8,
        downsample_rate: int = 1,
    ) -> None:
        super().__init__()
        self.embedding_dim = embedding_dim
        self.internal_dim = embedding_dim // downsample_rate # 全聯接層的縮放大小
        self.num_heads = num_heads
        assert self.internal_dim % num_heads == 0, "num_heads 必須整除 embedding_dim"

        # MLP dim => internal_dim
        self.q_proj = nn.Linear(embedding_dim, self.internal_dim)
        self.k_proj = nn.Linear(embedding_dim, self.internal_dim)
        self.v_proj = nn.Linear(embedding_dim, self.internal_dim)

        # MLP internal_dim => dim
        self.out_proj = nn.Linear(self.internal_dim, embedding_dim)

    def _separate_heads(self, x: Tensor, num_heads: int) -> Tensor:
        b, n, c = x.shape
        x = x.reshape(b, n, num_heads, c // num_heads)
        return x.transpose(1, 2)

    def _recombine_heads(self, x: Tensor) -> Tensor:
        b, n_heads, n_tokens, c_per_head = x.shape
        x = x.transpose(1, 2)
        return x.reshape(b, n_tokens, n_heads * c_per_head)  # B x N_tokens x C

    def forward(self, q: Tensor, k: Tensor, v: Tensor) -> Tensor:
        q = self.q_proj(q) # MLP
        k = self.k_proj(k) # MLP
        v = self.v_proj(v) # MLP

        # Separate into heads
        q = self._separate_heads(q, self.num_heads)
        k = self._separate_heads(k, self.num_heads)
        v = self._separate_heads(v, self.num_heads)

        # Attention
        _, _, _, c_per_head = q.shape
        attn = q @ k.permute(0, 1, 3, 2)  # B x N_heads x N_tokens x N_tokens
        attn = attn / math.sqrt(c_per_head) # 這種縮放操作可以防止當特徵數值很大時，點積的結果過大導致的梯度爆炸問題。
        attn = torch.softmax(attn, dim=-1) # 在最後一個維度做，對應於一個Q對所有K的相似度

        # Get output
        out = attn @ v
        out = self._recombine_heads(out)
        out = self.out_proj(out)
        return out
    
class MLPBlock(nn.Module):
    def __init__(
        self,
        embedding_dim: int,
        mlp_dim: int,
        act: Type[nn.Module] = nn.GELU,
    ) -> None:
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(embedding_dim, mlp_dim),
            act(),
            nn.Linear(mlp_dim, embedding_dim),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layers(x)

In [ ]:
# for testing purposes
image_embedding = torch.randn(1,256,64,64)
text_embedding = torch.randn(1,512)
trans = Text2ImageTransformer()
a, b = trans(image_embedding, text_embedding)
print(a.size(),b.size())
# TODO: postion embedding
# TODO: is GLOBAL QUERY needed?
# TODO: decoder study and implement

In [ ]:
import loralib as lora
lora.Linear(16, 24, r=16)

In [ ]:
class TextSAM(nn.Module):
    def __init__(self):
        super(TextSAM, self).__init__()
        self.image_enc = sam.image_encoder
        self.clip = clip_model
        self.linear = nn.Linear(256,512)
        self.transformer = nn.Transformer(d_model=512, nhead=8)

    def forward(self, x):
        x = self.image_enc(x)
        x = x.view(1,256,-1).permute(0,2,1)
        x = self.linear(x)
        xt = self.clip.encode_text(clip.tokenize(["lung"]).to(device))
        xt = xt.unsqueeze(0)
        # x = x.permute(1,0,2)
        # xt = xt.permute(1,0,2)
        print(x.size())
        print(xt.size())

        # x = self.transformer(x,xt)
        return x

model = TextSAM()
for batch in dataloader:
    images, labels = batch
    images = sam.preprocess(images)
    images = images.unsqueeze(0)
    print(images.size()) # [1, 3, 1024, 1024]
    model(images)
    break

# train

In [ ]:
import torch.optim as optim
num_epochs = 100
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
for epoch in range(num_epochs):
    for batch in dataloader:
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')